In [22]:
%load_ext autoreload
%autoreload 2

In [23]:
from magpylib import magnet
from nn_magnetics.utils.physics import demagnetizing_factor
from nn_magnetics.utils.metrics import angle_error, relative_amplitude_error
from nn_magnetics.data.create_data import generate_points_grid
import numpy as np

In [24]:
a = 1.28545749
b = 1.60580051

In [30]:
cuboid1 = magnet.Cuboid(dimension=(a, b, 1), polarization=(-1.37148570e-17, -2.83440378e-18,  9.21923442e-01))
cuboid1.susceptibility = (1, 1, 1)
cuboid2 = magnet.Cuboid(dimension=(a, b, 1), polarization=(0, 0, 0.92192344))

In [31]:
grid = generate_points_grid(26, 1, 1)

In [32]:
B1 = cuboid1.getB(grid)
B2 = cuboid2.getB(grid)

In [33]:
angle_err = np.mean(angle_error(B1, B2))
angle_err

np.float64(2.3374308008892997e-07)

In [34]:
amp_err = np.mean(relative_amplitude_error(B1, B2, True))
amp_err

np.float64(2.1693774697313602e-07)